In [85]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [86]:
from groq import Groq

### Lets Make SimpleAI Assistent Using Groq And DEEPSEEK

In [87]:
api_key = os.getenv("GROQ_API_KEY")

# Print it (for checking)
print("API Key:", api_key)

API Key: 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


### In this way you can directly call Groq From its SDK , but u want get chance to use RAG , AGENTS , kinda Tools Which help for Langraph


In [88]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="deepseek-r1-distill-llama-70b",
)

print(chat_completion.choices[0].message.content)

<think>
Okay, so I need to explain the importance of fast language models. Hmm, where do I start? I know that language models are a big part of AI these days, especially with things like ChatGPT and such. But why is speed important for them?

Well, first off, when you're interacting with a language model, you don't want to wait forever for a response. If it's slow, it can be frustrating. Imagine asking a question and waiting minutes for an answer. That's not very user-friendly. So, faster models probably make the experience smoother and more enjoyable.

Then there's the aspect of scalability. If a model is too slow, it might not handle a lot of users at once. Like, if a company is using a language model for customer service, a slow model could create a bottleneck, making the service less efficient. So, speed here would help serve more people without delays.

Another thought is about processing power. Maybe faster models are more efficient in terms of computational resources. If a model

### In this way You can use for Other Purposes ass well 

In [89]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="deepseek-r1-distill-llama-70b")


In [90]:
from langchain_core.output_parsers import StrOutputParser
answer = StrOutputParser()
question = "what is the best way to learn AI"
print(answer.invoke(llm.invoke(question)))

<think>
Okay, so I want to learn AI, but I'm not really sure where to start. I've heard a lot about AI being a big field, but I don't know the specifics. Maybe I should begin by figuring out what exactly AI entails. I know it's about machines learning and doing tasks that humans do, like understanding speech or recognizing images. But beyond that, I'm a bit fuzzy.

I think the first step is to get the basics right. I remember someone mentioning that math is important for AI. So, what math exactly? I've heard terms like linear algebra, calculus, and statistics being thrown around. I'm okay with algebra, but calculus and stats are a bit daunting. Maybe I should start by brushing up on those. Where can I find resources to learn them? Maybe there are online courses or textbooks.

Next, programming skills. I know a little Python, but I'm not super confident. I've heard that Python is widely used in AI, so maybe focusing on that is a good idea. I should probably practice more and maybe take 

### Lest give some Memory Idea to this Chat AI Assistent

In [91]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [92]:
store = {}

In [93]:

def get_session_history(session_id: str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [94]:
config = {"configurable" : {"session_id" : "first session"}}

In [95]:
new_question = "my name is gihan Lkmal"
model_with_memory = RunnableWithMessageHistory(llm , get_session_history)
new_memory_model = model_with_memory.invoke(new_question , config=config)
print(answer.invoke(new_memory_model))

<think>

</think>

Hello, Gihan Lakmal! How can I assist you today? 😊


In [96]:
new_question = "What is my name plz?"
model_with_memory = RunnableWithMessageHistory(llm , get_session_history)
new_memory_model = model_with_memory.invoke(new_question , config=config)
print(answer.invoke(new_memory_model))

<think>

Sure, your name is Gihan Lakmal! If you need anything else, feel free to ask.


#### lets learn RAG WITH LCEL now , then we can get into Langgraph and Its TOOLS 

In [97]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda

In [100]:
loader = DirectoryLoader('data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB



'from langchain.embeddings import HuggingFaceBgeEmbeddings\n\nmodel_name = "BAAI/bge-base-en-v1.5"\nmodel_kwargs = {\'device\': \'cpu\'}\nencode_kwargs = {\'normalize_embeddings\': True} # set True to compute cosine similarity\nembeddings = HuggingFaceBgeEmbeddings(\n    model_name=model_name,\n    model_kwargs=model_kwargs,\n    encode_kwargs=encode_kwargs,\n)\n'

In [101]:
db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

InternalError: Database error: error returned from database: (code: 1) no such table: tenants

In [ ]:
template = """Be nice and friendly with thew user and Answer the question based  on the following context:
{context} . 

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [57]:
store = {}

In [58]:
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [63]:
def rag_with_memory(session_id, question):
    # Retrieve the session history (if any)
    message_history = get_session_history(session_id)

    # Setup the retrieval chain
    retrieval_chain = (
        RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
        | prompt
        | llm
        | StrOutputParser()
    )
    
    # Invoke the retrieval chain and process the response
    response = retrieval_chain.invoke(question)
    
    # Store the question and response in the session history for future context
    message_history.add_user_message(question)
    message_history.add_ai_message(response)
    
    return response

# from here you can ask Questionsd about using the provided txt file

In [64]:
# Continuously ask questions until "bye" is input
session_id = "unique_session_1"

while True:
    question = input("Ask a question (type 'bye' to exit): ")
    
    # Check if the user wants to exit the loop
    if question.lower() == "bye":
        print("Goodbye!")
        break
    
    # Get the response using the RAG system with memory
    response = rag_with_memory(session_id, question)
    
    # Print the response
    print(f"Response: {response}")


Response: <think>
Okay, the user wrote "my nama is gihan." I notice that "nama" is probably a typo and they meant "name." So their name is Gihan. They might be introducing themselves or just sharing their name. I should respond in a friendly and welcoming manner. Maybe ask how I can assist them today to keep the conversation going.
</think>

Hello, Gihan! How can I assist you today?
Goodbye!


# Tools and Agent


In [65]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [66]:
api_wrapper=WikipediaAPIWrapper()

In [67]:
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [68]:
tool.name

'wikipedia'

In [69]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [70]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [71]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Model Context Protocol
Summary: The Model Context Protocol (MCP) is an open standard developed by the artificial intelligence company Anthropic for enabling large language model (LLM) applications to interact with external tools, systems, and data sources. Designed to standardize context exchange between AI assistants and software environments, MCP provides a model-agnostic interface for reading files, executing functions, and handling contextual prompts. It was officially announced and open-sourced by Anthropic in November 2024, with subsequent adoption by major AI providers including OpenAI and Google DeepMind.

Page: Retrieval-augmente

In [72]:
from langchain_community.tools import YouTubeSearchTool

In [73]:
tool2=YouTubeSearchTool()

In [74]:
tool2.name

'youtube_search'

In [75]:
tool2.run("ramuva")

"['https://www.youtube.com/watch?v=VNegZKkYBNg&pp=ygUGcmFtdXZh', 'https://www.youtube.com/watch?v=_YKOTDSuUu0&pp=ygUGcmFtdXZh']"

In [76]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [78]:
tool3 = TavilySearchResults()

ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [80]:
print(tool.run({"query": "What happened in the latest burning man floods"}))

Page: Burning Man
Summary: Burning Man is a week-long large-scale desert event focused on "community, art, self-expression, and self-reliance" held annually in the western United States. The event's name comes from its ceremony on the penultimate night of the event: the symbolic burning of a large wooden effigy, referred to as the Man, the Saturday evening before Labor Day. Since 1990, the event has been at Black Rock City in northwestern Nevada, a temporary city erected in the Black Rock Desert about 100 miles (160 km) north-northeast of Reno. According to Burning Man co-founder Larry Harvey in 2004, the event is guided by ten stated principles: radical inclusion, gifting, decommodification, radical self-reliance, radical self-expression, communal effort, civic responsibility, leaving no trace, participation, and immediacy.
Burning Man features no headliners or scheduled performers; participants create all the art, activities, and events. Artwork includes experimental and interactive 

In [81]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [82]:
tool=load_tools(["wikipedia"],llm=llm)

In [83]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\GIHAN LAKMAL\AppData\Local\Temp\ipykernel_52624\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [84]:
agent.run("What is current GDP of Srilanka?")

C:\Users\GIHAN LAKMAL\AppData\Local\Temp\ipykernel_52624\400833777.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is current GDP of Srilanka?")




> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `<think>
Okay, so I need to find the current GDP of Sri Lanka. I'm not entirely sure about the exact figure, but I remember that GDP stands for Gross Domestic Product, which is a measure of a country's economic performance. Since I don't have the most up-to-date information, I'll have to look it up.

First, I should consider where to find reliable data. I know that sources like the World Bank, International Monetary Fund (IMF), or the Central Bank of Sri Lanka provide accurate economic statistics. Alternatively, checking a recent Wikipedia article on Sri Lanka's economy might give me the latest figures.

Wait, the user mentioned that I can use the 'wikipedia' tool, so maybe I can search for "GDP of Sri Lanka" on Wikipedia. That should give me the most recent data availabl